<a href="https://colab.research.google.com/github/duartele/exerc-jupyternotebook/blob/main/SQL_course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To use this commands, we need to use Kaggle's notebook because a client is requiered.

Each Kaggle user can scan 5TB every 30 days for free. Once you hit that limit, you'll have to wait for it to reset.


## So I'm using this just to have a backup

In [1]:
from google.cloud import bigquery

In [ ]:
# Create a "Client" object
client = bigquery.Client()

In [ ]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [ ]:
# List all the tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are four!)
for table in tables:  
    print(table.table_id)

In [ ]:
# Construct a reference to the "full" table
table_ref = dataset_ref.table("full")

# API request - fetch the table
table = client.get_table(table_ref)

In [ ]:
# Print information on all the columns in the "full" table in the "hacker_news" dataset
table.schema

Each SchemaField tells us about a specific column (which we also refer to as a field). In order, the information is:

The name of the column
The field type (or datatype) in the column
The mode of the column ('NULLABLE' means that a column allows NULL values, and is the default)
A description of the data in that column
The first field has the SchemaField:

SchemaField('by', 'string', 'NULLABLE', "The username of the item's author.",())

In [ ]:
# Preview the first five lines of the "full" table
client.list_rows(table, max_results=5).to_dataframe()

In [ ]:
# Preview the first five entries in the "title" column of the "full" table
client.list_rows(table, selected_fields=table.schema[:1], max_results=5).to_dataframe()

In [ ]:
# Query to select all the items from the "city" column where the "country" column is 'US'
query = """
        SELECT city, country #more columns just use commas
        #SELECT DISTINCT city #if you don't want repetitions
        #SELECT * #for selecting all the columns
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

To run this query we need:

In [ ]:
# Create a "Client" object
client = bigquery.Client()
# Set up the query
query_job = client.query(query)
# API request - run the query, and return a pandas DataFrame
us_cities = query_job.to_dataframe()

# To check the size of a query before run it (for avoid exceeding the 5TB limit)

Estimate

In [ ]:
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, job_config=dry_run_config)

print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

Put a Limit (1MB in this case)

In [ ]:
# Only run the query if it's less than 1 MB
ONE_MB = 1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_MB)

# Set up the query (will only run if it's less than 1 MB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
safe_query_job.to_dataframe()

## Agregate functions
COUNT() SUM() AVG() MIN() MAX()

HAVING is used in combination with GROUP BY to ignore groups that don't meet certain criteria

In [ ]:
#SELECT Animal, COUNT(ID)
#COUNT(1) YOU CAN USE COUNT(1) - MORE FAST AND EASY TO READ - COUNT EACH GROUP
#FROM `table`
#WHERE deleted = True
#GROUP BY Animal
#HAVING COUNT(ID) > 1
#HAVING COUNT(1) > 1
#ORDER BY Animal
#ORDER BY Animal DESC

# DATE = YYYY-MM-DD

We can use the function EXTRACT

In [ ]:
#SELECT Animal, EXTRACT(DAY from Date) as Day
#SELECT Animal, EXTRACT(WEEK from Date) as Week- 1 is Sunday and 7 is Saturday

# 'AS' is used to Aliasing variables
# Now we are taking a look in the  WITH...AS command

WITH...AS create a temporary table (called CTE - Common Table Expression)


In [ ]:
query_with_CTE = """ 
                 WITH time AS 
                 (
                     SELECT DATE(block_timestamp) AS trans_date
                     FROM `bigquery-public-data.crypto_bitcoin.transactions`
                 )
                 SELECT COUNT(1) AS transactions,
                        trans_date
                 FROM time
                 GROUP BY trans_date
                 ORDER BY trans_date
                 """

#Join

INNER JOIN is used when a row will only be put in the final output table if the value in the columns you're using to combine them shows up in both the tables you're joining

In [ ]:
query = """
        SELECT L.license, COUNT(1) AS number_of_files
        FROM `bigquery-public-data.github_repos.sample_files` AS sf
        INNER JOIN `bigquery-public-data.github_repos.licenses` AS L 
            ON sf.repo_name = L.repo_name
        GROUP BY L.license
        ORDER BY number_of_files DESC
        """
